In [2]:
%load_ext autoreload
%autoreload 2

import os
import sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

from bs4 import BeautifulSoup
import requests
import pandas as pd
import src
from src import object_test
from src import open_page as op

import psycopg2
from sqlalchemy import create_engine

from src import local

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


First I want a list of all events on ufcstats.com

In [3]:
bouts = pd.read_csv('../data/ufcstats_bouts.csv', index_col=0)
bouts

FileNotFoundError: [Errno 2] File ../data/ufcstats_bouts.csv does not exist: '../data/ufcstats_bouts.csv'

In [4]:
bout_link = bouts['0'][0]
bout_link

'http://www.ufcstats.com/fight-details/f8dd1e75978a3957'

In [5]:
bout_page = op.open_link(bout_link, async_=True)
list_of_tables = pd.read_html(bout_page)
len(list_of_tables)

New Ip Address 46.166.139.111


4

Now I want all the stats.

In [6]:
general = list_of_tables[1].copy()
general['round'] = general.index+1

general.columns = [column[0].strip().replace(' ', '_').replace('%', 'prcnt').replace('.', '') for column in general.columns]

general

,Fighter,KD,Sig_str,Sig_str_prcnt,Total_str,Td_prcnt,Td_prcnt,Sub_att,Pass,Rev,round
0,Dustin Poirier Dan Hooker,0 0,20 of 31 33 of 67,64% 49%,20 of 31 35 of 69,0 of 0 0 of 0,0% 0%,0 0,0 0,0 0,1
1,Dustin Poirier Dan Hooker,0 0,48 of 83 66 of 98,57% 67%,53 of 88 69 of 101,0 of 6 0 of 0,0% 0%,0 0,0 0,0 0,2
2,Dustin Poirier Dan Hooker,0 0,28 of 40 20 of 39,70% 51%,38 of 50 30 of 51,0 of 0 1 of 1,0% 100%,1 0,0 0,0 0,3
3,Dustin Poirier Dan Hooker,0 0,20 of 28 11 of 14,71% 78%,43 of 53 23 of 27,1 of 1 2 of 2,100% 100%,3 0,1 0,0 0,4
4,Dustin Poirier Dan Hooker,0 0,37 of 49 25 of 35,75% 71%,54 of 67 25 of 35,0 of 0 1 of 6,0% 16%,0 0,0 0,0 0,5


# Scripted

## Strikes

In [7]:
from src import scrape

In [8]:
list_of_tables[3]

,Fighter,Sig. str,Sig. str. %,Head,Body,Leg,Distance,Clinch,Ground,Unnamed: 9_level_0
,Round 1,Round 1,Round 1,Round 1,Round 1,Round 1,Round 1,Round 1,Round 1,Round 1
,Round 2,Round 2,Round 2,Round 2,Round 2,Round 2,Round 2,Round 2,Round 2,Round 2
,Round 3,Round 3,Round 3,Round 3,Round 3,Round 3,Round 3,Round 3,Round 3,Round 3
,Round 4,Round 4,Round 4,Round 4,Round 4,Round 4,Round 4,Round 4,Round 4,Round 4
,Round 5,Round 5,Round 5,Round 5,Round 5,Round 5,Round 5,Round 5,Round 5,Round 5
0,Dustin Poirier Dan Hooker,20 of 31 33 of 67,64% 49%,11 of 20 14 of 42,8 of 10 10 of 16,1 of 1 9 of 9,16 of 27 29 of 62,4 of 4 4 of 5,0 of 0 0 of 0,NaN
1,Dustin Poirier Dan Hooker,48 of 83 66 of 98,57% 67%,43 of 77 47 of 73,4 of 5 12 of 18,1 of 1 7 of 7,37 of 70 60 of 89,11 of 13 6 of 9,0 of 0 0 of 0,NaN
2,Dustin Poirier Dan Hooker,28 of 40 20 of 39,70% 51%,25 of 36 15 of 30,3 of 4 3 of 5,0 of 0 2 of 4,25 of 36 15 of 33,1 of 1 1 of 1,2 of 3 4 of 5,NaN
3,Dustin Poirier Dan Hooker,20 of 28 11 of 14,71% 78%,17 of 25 5 of 8,3 of 3 5 of 5,0 of 0 1 of 1,9 of 14 9 of 12,0 of 0 0 of 0,11 of 14 2 of 2,NaN
4,Dustin Poirier Dan Hooker,37 of 49 25 of 35,75% 71%,35 of 47 17 of 24,2 of 2 6 of 9,0 of 0 2 of 2,29 of 41 23 of 33,4 of 4 1 of 1,4 of 4 1 of 1,NaN


In [9]:
strikes = list_of_tables[3]

strikes = scrape.format_stats_table(strikes)

strikes

,fighter,sig_str,sig_str_prcnt,head,body,leg,distance,clinch,ground,unnamed:_9_level_0,round
0,Dustin Poirier Dan Hooker,20 of 31 33 of 67,64% 49%,11 of 20 14 of 42,8 of 10 10 of 16,1 of 1 9 of 9,16 of 27 29 of 62,4 of 4 4 of 5,0 of 0 0 of 0,NaN,1
1,Dustin Poirier Dan Hooker,48 of 83 66 of 98,57% 67%,43 of 77 47 of 73,4 of 5 12 of 18,1 of 1 7 of 7,37 of 70 60 of 89,11 of 13 6 of 9,0 of 0 0 of 0,NaN,2
2,Dustin Poirier Dan Hooker,28 of 40 20 of 39,70% 51%,25 of 36 15 of 30,3 of 4 3 of 5,0 of 0 2 of 4,25 of 36 15 of 33,1 of 1 1 of 1,2 of 3 4 of 5,NaN,3
3,Dustin Poirier Dan Hooker,20 of 28 11 of 14,71% 78%,17 of 25 5 of 8,3 of 3 5 of 5,0 of 0 1 of 1,9 of 14 9 of 12,0 of 0 0 of 0,11 of 14 2 of 2,NaN,4
4,Dustin Poirier Dan Hooker,37 of 49 25 of 35,75% 71%,35 of 47 17 of 24,2 of 2 6 of 9,0 of 0 2 of 2,29 of 41 23 of 33,4 of 4 1 of 1,4 of 4 1 of 1,NaN,5


In [10]:
bout_id = bout_link.split('/')[-1]
strikes['bout_id'] = bout_id

In [11]:
general

,Fighter,KD,Sig_str,Sig_str_prcnt,Total_str,Td_prcnt,Td_prcnt,Sub_att,Pass,Rev,round
0,Dustin Poirier Dan Hooker,0 0,20 of 31 33 of 67,64% 49%,20 of 31 35 of 69,0 of 0 0 of 0,0% 0%,0 0,0 0,0 0,1
1,Dustin Poirier Dan Hooker,0 0,48 of 83 66 of 98,57% 67%,53 of 88 69 of 101,0 of 6 0 of 0,0% 0%,0 0,0 0,0 0,2
2,Dustin Poirier Dan Hooker,0 0,28 of 40 20 of 39,70% 51%,38 of 50 30 of 51,0 of 0 1 of 1,0% 100%,1 0,0 0,0 0,3
3,Dustin Poirier Dan Hooker,0 0,20 of 28 11 of 14,71% 78%,43 of 53 23 of 27,1 of 1 2 of 2,100% 100%,3 0,1 0,0 0,4
4,Dustin Poirier Dan Hooker,0 0,37 of 49 25 of 35,75% 71%,54 of 67 25 of 35,0 of 0 1 of 6,0% 16%,0 0,0 0,0 0,5


In [12]:
bout_soup = BeautifulSoup(bout_page)

outcomes = bout_soup.find_all(class_='b-fight-details__person')

outcomes = [outcome.find('i').get_text().strip() for outcome in outcomes]

outcomes = ' '.join(outcomes)

strikes['outcome'] = outcomes
strikes

,fighter,sig_str,sig_str_prcnt,head,body,leg,distance,clinch,ground,unnamed:_9_level_0,round,bout_id,outcome
0,Dustin Poirier Dan Hooker,20 of 31 33 of 67,64% 49%,11 of 20 14 of 42,8 of 10 10 of 16,1 of 1 9 of 9,16 of 27 29 of 62,4 of 4 4 of 5,0 of 0 0 of 0,NaN,1,f8dd1e75978a3957,W L
1,Dustin Poirier Dan Hooker,48 of 83 66 of 98,57% 67%,43 of 77 47 of 73,4 of 5 12 of 18,1 of 1 7 of 7,37 of 70 60 of 89,11 of 13 6 of 9,0 of 0 0 of 0,NaN,2,f8dd1e75978a3957,W L
2,Dustin Poirier Dan Hooker,28 of 40 20 of 39,70% 51%,25 of 36 15 of 30,3 of 4 3 of 5,0 of 0 2 of 4,25 of 36 15 of 33,1 of 1 1 of 1,2 of 3 4 of 5,NaN,3,f8dd1e75978a3957,W L
3,Dustin Poirier Dan Hooker,20 of 28 11 of 14,71% 78%,17 of 25 5 of 8,3 of 3 5 of 5,0 of 0 1 of 1,9 of 14 9 of 12,0 of 0 0 of 0,11 of 14 2 of 2,NaN,4,f8dd1e75978a3957,W L
4,Dustin Poirier Dan Hooker,37 of 49 25 of 35,75% 71%,35 of 47 17 of 24,2 of 2 6 of 9,0 of 0 2 of 2,29 of 41 23 of 33,4 of 4 1 of 1,4 of 4 1 of 1,NaN,5,f8dd1e75978a3957,W L


In [13]:
strikes = strikes.drop(labels = 'unnamed:_9_level_0', axis=1)
strikes

,fighter,sig_str,sig_str_prcnt,head,body,leg,distance,clinch,ground,round,bout_id,outcome
0,Dustin Poirier Dan Hooker,20 of 31 33 of 67,64% 49%,11 of 20 14 of 42,8 of 10 10 of 16,1 of 1 9 of 9,16 of 27 29 of 62,4 of 4 4 of 5,0 of 0 0 of 0,1,f8dd1e75978a3957,W L
1,Dustin Poirier Dan Hooker,48 of 83 66 of 98,57% 67%,43 of 77 47 of 73,4 of 5 12 of 18,1 of 1 7 of 7,37 of 70 60 of 89,11 of 13 6 of 9,0 of 0 0 of 0,2,f8dd1e75978a3957,W L
2,Dustin Poirier Dan Hooker,28 of 40 20 of 39,70% 51%,25 of 36 15 of 30,3 of 4 3 of 5,0 of 0 2 of 4,25 of 36 15 of 33,1 of 1 1 of 1,2 of 3 4 of 5,3,f8dd1e75978a3957,W L
3,Dustin Poirier Dan Hooker,20 of 28 11 of 14,71% 78%,17 of 25 5 of 8,3 of 3 5 of 5,0 of 0 1 of 1,9 of 14 9 of 12,0 of 0 0 of 0,11 of 14 2 of 2,4,f8dd1e75978a3957,W L
4,Dustin Poirier Dan Hooker,37 of 49 25 of 35,75% 71%,35 of 47 17 of 24,2 of 2 6 of 9,0 of 0 2 of 2,29 of 41 23 of 33,4 of 4 1 of 1,4 of 4 1 of 1,5,f8dd1e75978a3957,W L


## general

In [14]:
general = list_of_tables[1]
general = scrape.format_stats_table(general)

general

,fighter,kd,sig_str,sig_str_prcnt,total_str,td_prcnt,td_prcnt,sub_att,pass,rev,round
0,Dustin Poirier Dan Hooker,0 0,20 of 31 33 of 67,64% 49%,20 of 31 35 of 69,0 of 0 0 of 0,0% 0%,0 0,0 0,0 0,1
1,Dustin Poirier Dan Hooker,0 0,48 of 83 66 of 98,57% 67%,53 of 88 69 of 101,0 of 6 0 of 0,0% 0%,0 0,0 0,0 0,2
2,Dustin Poirier Dan Hooker,0 0,28 of 40 20 of 39,70% 51%,38 of 50 30 of 51,0 of 0 1 of 1,0% 100%,1 0,0 0,0 0,3
3,Dustin Poirier Dan Hooker,0 0,20 of 28 11 of 14,71% 78%,43 of 53 23 of 27,1 of 1 2 of 2,100% 100%,3 0,1 0,0 0,4
4,Dustin Poirier Dan Hooker,0 0,37 of 49 25 of 35,75% 71%,54 of 67 25 of 35,0 of 0 1 of 6,0% 16%,0 0,0 0,0 0,5


In [15]:
bout_id = bout_link.split('/')[-1]

general['bout_id'] = bout_id

In [16]:
general

,fighter,kd,sig_str,sig_str_prcnt,total_str,td_prcnt,td_prcnt,sub_att,pass,rev,round,bout_id
0,Dustin Poirier Dan Hooker,0 0,20 of 31 33 of 67,64% 49%,20 of 31 35 of 69,0 of 0 0 of 0,0% 0%,0 0,0 0,0 0,1,f8dd1e75978a3957
1,Dustin Poirier Dan Hooker,0 0,48 of 83 66 of 98,57% 67%,53 of 88 69 of 101,0 of 6 0 of 0,0% 0%,0 0,0 0,0 0,2,f8dd1e75978a3957
2,Dustin Poirier Dan Hooker,0 0,28 of 40 20 of 39,70% 51%,38 of 50 30 of 51,0 of 0 1 of 1,0% 100%,1 0,0 0,0 0,3,f8dd1e75978a3957
3,Dustin Poirier Dan Hooker,0 0,20 of 28 11 of 14,71% 78%,43 of 53 23 of 27,1 of 1 2 of 2,100% 100%,3 0,1 0,0 0,4,f8dd1e75978a3957
4,Dustin Poirier Dan Hooker,0 0,37 of 49 25 of 35,75% 71%,54 of 67 25 of 35,0 of 0 1 of 6,0% 16%,0 0,0 0,0 0,5,f8dd1e75978a3957


In [17]:
bout_soup = BeautifulSoup(bout_page)
outcomes = bout_soup.find_all(class_='b-fight-details__person')
outcomes = [outcome.find('i').get_text().strip() for outcome in outcomes]

outcomes = ' '.join(outcomes)

general['outcome'] = outcomes
general

,fighter,kd,sig_str,sig_str_prcnt,total_str,td_prcnt,td_prcnt,sub_att,pass,rev,round,bout_id,outcome
0,Dustin Poirier Dan Hooker,0 0,20 of 31 33 of 67,64% 49%,20 of 31 35 of 69,0 of 0 0 of 0,0% 0%,0 0,0 0,0 0,1,f8dd1e75978a3957,W L
1,Dustin Poirier Dan Hooker,0 0,48 of 83 66 of 98,57% 67%,53 of 88 69 of 101,0 of 6 0 of 0,0% 0%,0 0,0 0,0 0,2,f8dd1e75978a3957,W L
2,Dustin Poirier Dan Hooker,0 0,28 of 40 20 of 39,70% 51%,38 of 50 30 of 51,0 of 0 1 of 1,0% 100%,1 0,0 0,0 0,3,f8dd1e75978a3957,W L
3,Dustin Poirier Dan Hooker,0 0,20 of 28 11 of 14,71% 78%,43 of 53 23 of 27,1 of 1 2 of 2,100% 100%,3 0,1 0,0 0,4,f8dd1e75978a3957,W L
4,Dustin Poirier Dan Hooker,0 0,37 of 49 25 of 35,75% 71%,54 of 67 25 of 35,0 of 0 1 of 6,0% 16%,0 0,0 0,0 0,5,f8dd1e75978a3957,W L


In [27]:
engine = create_engine(f'postgresql://{local.user}:{local.password}@{local.host}:{local.port}/ufc_odds')


In [28]:
strikes.to_sql('test', engine, if_exists='append', index=False)

In [1]:
general['bout_id'] = bout_id
general['outcome'] = outcomes
general.to_sql('general', engine, if_exists='append', index=False)

NameError: name 'bout_id' is not defined

# Scraping

In [40]:
engine = create_engine(f'postgresql://{local.user}:{local.password}@{local.host}:{local.port}/ufc_odds')


for bout_link in bouts['0'][94:]:
    print('--------------------------------')
    print(bout_link)
    print('--------------------------------')

    # open page and parse tables with pd
    bout_page = op.open_link(bout_link, async_=True)
    list_of_tables = pd.read_html(bout_page)
    
    # Open page with beautiful soup and get outcomes info
    bout_soup = BeautifulSoup(bout_page)
    outcomes = bout_soup.find_all(class_='b-fight-details__person')
    outcomes = [outcome.find('i').get_text().strip() for outcome in outcomes]
    outcomes = ' '.join(outcomes)
    
    #get bout id from link
    bout_id = bout_link.split('/')[-1]

    
    ## Get tables and send to postgres
    #strikes
    strikes = list_of_tables[3]
    strikes = scrape.format_stats_table(strikes)

    strikes['bout_id'] = bout_id
    strikes['outcome'] = outcomes
    strikes = strikes.drop(labels = 'unnamed:_9_level_0', axis=1)
    strikes.to_sql('strikes', engine, if_exists='append', index=False)

    ## general

    general = list_of_tables[1]
    general = scrape.format_stats_table(general)

    general['bout_id'] = bout_id
    general['outcome'] = outcomes
    general.to_sql('general', engine, if_exists='append', index=False)



    

--------------------------------
http://www.ufcstats.com/fight-details/9fc6aba53508dc48
--------------------------------
New Ip Address 18.27.197.252
--------------------------------
http://www.ufcstats.com/fight-details/e2503ed6a1f9257d
--------------------------------
New Ip Address 18.27.197.252
--------------------------------
http://www.ufcstats.com/fight-details/2a1826612a7d739c
--------------------------------
New Ip Address 18.27.197.252
--------------------------------
http://www.ufcstats.com/fight-details/1a2d13834696c4ab
--------------------------------
New Ip Address 18.27.197.252
--------------------------------
http://www.ufcstats.com/fight-details/ef526567c4dcf371
--------------------------------
New Ip Address 18.18.248.17
--------------------------------
http://www.ufcstats.com/fight-details/b46f2f007b622bce
--------------------------------
New Ip Address 18.18.248.17
--------------------------------
http://www.ufcstats.com/fight-details/cf0a6466b762c668
--------------

KeyboardInterrupt: 

In [42]:
bout_link = bouts['0'][0]

In [48]:
general_1

,fighter,kd,sig_str,sig_str_prcnt,total_str,td_count,td_prcnt,sub_att,pass,rev,round,bout_id,outcome
0,Dan Hooker,0,33 of 67,49%,35 of 69,0 of 0,0%,0,0,0,1,f8dd1e75978a3957,L
1,Dan Hooker,0,66 of 98,67%,69 of 101,0 of 0,0%,0,0,0,2,f8dd1e75978a3957,L
2,Dan Hooker,0,20 of 39,51%,30 of 51,1 of 1,100%,0,0,0,3,f8dd1e75978a3957,L
3,Dan Hooker,0,11 of 14,78%,23 of 27,2 of 2,100%,0,0,0,4,f8dd1e75978a3957,L
4,Dan Hooker,0,25 of 35,71%,25 of 35,1 of 6,16%,0,0,0,5,f8dd1e75978a3957,L


In [26]:
bout_link = 'http://www.ufcstats.com/fight-details/e4fe950846b51bdf'

In [27]:
bout_page = op.open_link(bout_link)

New Ip Address 185.220.101.157


In [28]:
bout_soup = BeautifulSoup(bout_page)

In [33]:
list_of_tables[1]

,fighter,kd,sig_str,sig_str_prcnt,total_str,td_count,td_prcnt,sub_att,pass,rev,round,bout_id,outcome
0,Dustin Poirier Dan Hooker,0 0,20 of 31 33 of 67,64% 49%,20 of 31 35 of 69,0 of 0 0 of 0,0% 0%,0 0,0 0,0 0,1,f8dd1e75978a3957,W L
1,Dustin Poirier Dan Hooker,0 0,48 of 83 66 of 98,57% 67%,53 of 88 69 of 101,0 of 6 0 of 0,0% 0%,0 0,0 0,0 0,2,f8dd1e75978a3957,W L
2,Dustin Poirier Dan Hooker,0 0,28 of 40 20 of 39,70% 51%,38 of 50 30 of 51,0 of 0 1 of 1,0% 100%,1 0,0 0,0 0,3,f8dd1e75978a3957,W L
3,Dustin Poirier Dan Hooker,0 0,20 of 28 11 of 14,71% 78%,43 of 53 23 of 27,1 of 1 2 of 2,100% 100%,3 0,1 0,0 0,4,f8dd1e75978a3957,W L
4,Dustin Poirier Dan Hooker,0 0,37 of 49 25 of 35,75% 71%,54 of 67 25 of 35,0 of 0 1 of 6,0% 16%,0 0,0 0,0 0,5,f8dd1e75978a3957,W L


In [36]:
def split(fighter_index, value):
    if type(value) != str:
        return value

    split_string = value.split(' ')
    length = len(split_string)
    if  length > 1:
        if fighter_index == 0:
            fighter_0_stats = split_string[:int(length/2)]
            return ' '.join(fighter_0_stats)
        elif fighter_index == 1:
            fighter_1_stats = split_string[int(length/2):]
            return ' '.join(fighter_1_stats)
    else:
        return value
        


In [40]:
general.applymap(lambda x: split(1, x))

,fighter,kd,sig_str,sig_str_prcnt,total_str,td_count,td_prcnt,sub_att,pass,rev,round,bout_id,outcome
0,Dan Hooker,0,33 of 67,49%,35 of 69,0 of 0,0%,0,0,0,1,f8dd1e75978a3957,L
1,Dan Hooker,0,66 of 98,67%,69 of 101,0 of 0,0%,0,0,0,2,f8dd1e75978a3957,L
2,Dan Hooker,0,20 of 39,51%,30 of 51,1 of 1,100%,0,0,0,3,f8dd1e75978a3957,L
3,Dan Hooker,0,11 of 14,78%,23 of 27,2 of 2,100%,0,0,0,4,f8dd1e75978a3957,L
4,Dan Hooker,0,25 of 35,71%,25 of 35,1 of 6,16%,0,0,0,5,f8dd1e75978a3957,L


In [38]:
bouts = pd.read_csv('../data/ufcstats_bouts.csv', index_col=0)
bouts.to_csv('../data/remaining_bouts.csv')

In [39]:
bouts

,0
0,http://www.ufcstats.com/fight-details/f8dd1e75...
1,http://www.ufcstats.com/fight-details/f2d98149...
2,http://www.ufcstats.com/fight-details/9ce56aca...
3,http://www.ufcstats.com/fight-details/cfd47fdd...
4,http://www.ufcstats.com/fight-details/67ed3ad5...
...,...
5646,http://www.ufcstats.com/fight-details/ac7ca2ec...
5647,http://www.ufcstats.com/fight-details/46acd54c...
5648,http://www.ufcstats.com/fight-details/cecdc0da...
5649,http://www.ufcstats.com/fight-details/2d2bbc86...


In [65]:
last_bout_index = bouts[bouts['0']==bout_link].index[0]
bouts.loc[last_bout_index:].to_csv('../data/remaining_bouts.csv')

IndexError: index 0 is out of bounds for axis 0 with size 0

In [67]:
bouts = pd.read_csv('../data/ufcstats_bouts.csv', index_col=0)
bouts.to_csv('../data/remaining_bouts.csv')

In [4]:
engine = create_engine(f'postgresql://{local.user}:{local.password}@{local.host}:{local.port}/ufc_odds')

bouts = pd.read_sql("""
                    SELECT * FROM general;
                    """,
                    engine
                   )

In [5]:
bouts

,fighter,kd,sig_str,sig_str_prcnt,total_str,td_count,td_prcnt,sub_att,pass,rev,round,bout_id,outcome
0,Dustin Poirier,0,20 of 31,64%,20 of 31,0 of 0,0%,0,0,0,1,f8dd1e75978a3957,W
1,Dustin Poirier,0,48 of 83,57%,53 of 88,0 of 6,0%,0,0,0,2,f8dd1e75978a3957,W
2,Dustin Poirier,0,28 of 40,70%,38 of 50,0 of 0,0%,1,0,0,3,f8dd1e75978a3957,W
3,Dustin Poirier,0,20 of 28,71%,43 of 53,1 of 1,100%,3,1,0,4,f8dd1e75978a3957,W
4,Dustin Poirier,0,37 of 49,75%,54 of 67,0 of 0,0%,0,0,0,5,f8dd1e75978a3957,W
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25987,Art Jimmerson,0,0 of 0,0%,0 of 0,0 of 0,0%,0,0,0,1,cecdc0da584274b9,L
25988,Kevin Rosier,2,15 of 27,55%,38 of 53,0 of 0,0%,0,0,0,1,2d2bbc86e941e05c,W
25989,Zane Frazier,0,12 of 28,42%,13 of 29,0 of 0,0%,0,0,0,1,2d2bbc86e941e05c,L
25990,Gerard Gordeau,0,3 of 5,60%,3 of 5,0 of 0,0%,0,0,0,1,567a09fd200cfa05,W


# Fighter Links

In [17]:
bout_page = op.open_link(bout_link, async_=True)
bout_soup = BeautifulSoup(bout_page)

New Ip Address 109.70.100.26


In [20]:
name_elems = bout_soup.find_all(class_='b-fight-details__person-name')
links = [elem.find('a').get('href') for elem in name_elems]
links

['http://www.ufcstats.com/fighter-details/029eaff01e6bb8f0',
 'http://www.ufcstats.com/fighter-details/193b9d1858bc4df3']

# bouts table

In [88]:
title = bout_soup.find(class_='b-content__title')
event_link = title.find('a').get('href')

In [89]:
bouts = object_test.html_i_to_df(bout_soup, list_class="b-fight-details__text")
bouts['event_link'] = event_link
bouts

,Method,Round,Time,Timeformat,Referee,event_link
0,Decision - Unanimous,5,5:00,5 Rnd (5-5-5-5-5),Herb Dean,http://www.ufcstats.com/event-details/c32eab6c...


In [90]:
details_elem = bout_soup.find_all(class_='b-fight-details__text')[1]
detail_items = details_elem.find_all('i', recursive=False)

In [91]:
detail_items = [item.get_text().split('\n') for item in detail_items]
detail_items

[['', '', '          Details:', '        ', ''],
 ['',
  'Eric Colon ',
  '',
  '            ',
  '            ',
  '            48 - 46.',
  '          '],
 ['',
  'Derek Cleary ',
  '',
  '            ',
  '            ',
  '            48 - 47.',
  '          '],
 ['',
  "Sal D'amato ",
  '',
  '            ',
  '            ',
  '            48 - 47.',
  '          ']]

In [92]:
details = []
for item in detail_items:
    cleaned = [part.strip() for part in item]
    details.append(''.join(cleaned))

In [93]:
details = ''.join(details)

details = details[8:]
details

"Eric Colon48 - 46.Derek Cleary48 - 47.Sal D'amato48 - 47."

In [95]:
bouts['details'] = details

bouts

,Method,Round,Time,Timeformat,Referee,event_link,details
0,Decision - Unanimous,5,5:00,5 Rnd (5-5-5-5-5),Herb Dean,http://www.ufcstats.com/event-details/c32eab6c...,Eric Colon48 - 46.Derek Cleary48 - 47.Sal D'am...


In [96]:
def parse_details(bout_soup):    
    """
    input: beautiful soup of a ufcstats fight details page
    output: a parsed string of the 'details' item
    """
    #get all detail items
    details_elem = bout_soup.find_all(class_='b-fight-details__text')[1]
    detail_items = details_elem.find_all('i', recursive=False)
    #remove whitespace
    detail_items = [item.get_text().split('\n') for item in detail_items]
    details = []
    for item in detail_items:
        cleaned = [part.strip() for part in item]
        details.append(''.join(cleaned))
    details = ''.join(details)
    #remove label
    details = details[8:]
    return details

In [103]:
def scrape_bout_page(bout_soup, bout_link):
    #get event link
    title = bout_soup.find(class_='b-content__title')
    event_link = title.find('a').get('href')
    #get details
    details = parse_details(bout_soup)
    #get rest of fight info
    bouts = object_test.html_i_to_df(bout_soup, list_class="b-fight-details__text")
    bouts['details'] = details
    bouts['event_link'] = event_link
    bouts['link'] = bout_link
    return bouts

In [104]:
scrape_bout_page(bout_soup, bout_link)

,Method,Round,Time,Timeformat,Referee,details,event_link,link
0,Decision - Unanimous,5,5:00,5 Rnd (5-5-5-5-5),Herb Dean,Eric Colon48 - 46.Derek Cleary48 - 47.Sal D'am...,http://www.ufcstats.com/event-details/c32eab6c...,http://www.ufcstats.com/fight-details/f8dd1e75...
